In [58]:
import gurobipy as gp
import time
import pandas as pd
import numpy as np
import datetime


In [59]:
tipo_pontos = "9Pontos_e_1Deposito"


matrizz = pd.read_csv(f'../dados/Dados Gerados (testes)/{tipo_pontos}/distancia.csv', header=None)
demandass = pd.read_csv(f'../dados/Dados Gerados (testes)/{tipo_pontos}/demanda.csv', header=None)

In [60]:
V = list(range(1, len(demandass[0])))
V_0 = [0] + V
n = len(V)  # numero de clientes
Q = 500  # capacidade
MD = 7000  # Distancia Maxima para todo k
K = 2  # numero de veiculos disponiveis


In [61]:
Distance_Matrix = matrizz.to_numpy()
Demandas = {}
Demandas['coleta'] = demandass[0].tolist()
Demandas['entregas'] = demandass[1].tolist()
veiculos = [i + 1 for i in range(K)]


In [62]:
m = gp.Model("VRPSPD")


In [63]:
# variaveis
x = m.addVars(V_0, V_0, veiculos, vtype=gp.GRB.BINARY, name="x")
y = m.addVars(V_0, V_0, vtype=gp.GRB.INTEGER, name="y")  # coleta
z = m.addVars(V_0, V_0, vtype=gp.GRB.INTEGER, name="z")  # entrega


In [64]:
obj = gp.LinExpr()
for k in veiculos:
    for i in V_0:
        for j in V_0:
            obj += Distance_Matrix[i][j] * x[i, j, k]


In [65]:
# funcao objetivo
m.setObjective(obj, sense=gp.GRB.MINIMIZE)


In [66]:
# restrições
r1 = m.addConstrs(
    (gp.quicksum(x[i, j, k] for i in V_0 for k in veiculos) == 1 for j in V), name="X_ijk_eq_1"
)
r2 = m.addConstrs(
    (
        (gp.quicksum(x[i, j, k] for i in V_0)) -
        (gp.quicksum(x[j, i, k] for i in V_0)) == 0
        for j in V_0
        for k in veiculos
    ), name="chega_e_sai"
)
r3 = m.addConstrs(
    (
        gp.quicksum(x[0, j, k] for j in V) <= 1
        for k in veiculos
    ),
    name="max_k_veiculos"
)
r4 = m.addConstrs(
    (
        gp.quicksum(Distance_Matrix[i][j]*x[i, j, k]
                    for i in V_0 for j in V_0) <= MD
        for k in veiculos
    ),
    name="distancia_max_k"
)
r5 = m.addConstrs(
    (
        gp.quicksum(y[j, i] for i in V_0) - gp.quicksum(y[i, j] for i in V_0)
        == Demandas['coleta'][j] for j in V_0 if j != 0
    ),
    name="fluxo_y"
)
r6 = m.addConstrs(
    (
        gp.quicksum(z[i, j] for i in V_0) - gp.quicksum(z[j, i] for i in V_0)
        == Demandas['entregas'][j] for j in V_0 if j != 0
    ),
    name="fluxo_z"
)
r7 = m.addConstrs(
    (
        y[i, j] + z[i, j] <= Q*gp.quicksum(x[i, j, k] for k in veiculos)
        for i in V_0
        for j in V_0
    ),
    name="capacidade"
)


In [67]:
m.setParam('TimeLimit', 60)  # 60 segundos
m.update()
data_atual = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
m.write(f'../log/{data_atual}_VRPSPD.lp')


Set parameter TimeLimit to value 60


In [68]:

start_time = time.time()
m.optimize()
end_time = time.time()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 151 rows, 400 columns and 1462 nonzeros
Model fingerprint: 0x7597aa16
Variable types: 0 continuous, 400 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [6e+00, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+03]
Presolve removed 10 rows and 40 columns
Presolve time: 0.01s
Presolved: 141 rows, 360 columns, 1388 nonzeros
Variable types: 0 continuous, 360 integer (180 binary)

Root relaxation: objective 3.358356e+03, 164 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3358.35600    0   17          - 3358.35600

In [69]:
total_time = m.getAttr("Runtime")
print("Tempo total de execução do modelo: ", total_time)
print("Tempo real de execução: ", end_time - start_time)


Tempo total de execução do modelo:  0.33500003814697266
Tempo real de execução:  0.3386821746826172


In [70]:
if m.status == gp.GRB.Status.TIME_LIMIT:
    print('Tempo limite atingido. Melhor solução encontrada até agora: ', m.ObjVal)
else:
    print(m.ObjVal)

5012.0


In [76]:
m

<gurobi.Model MIP instance VRPSPD: 151 constrs, 400 vars, Parameter changes: TimeLimit=60.0, Username=(user-defined)>